<a href="https://colab.research.google.com/github/kulkarnisatishp/genai-course/blob/main/Open_AI_Swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [2]:
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-p19unoo4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-p19unoo4
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 27.5 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=5377b0c63f1c3

In [7]:
from google.colab import userdata
openai_api_key = userdata.get('openai_key')
news_api_key = userdata.get('newsapi_key')

In [4]:
import os
from swarm import Swarm, Agent
import requests
from IPython.display import display, Markdown

In [8]:
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['NEWS_API_KEY'] = news_api_key

In [9]:
greeting_agent = Agent(
    name="Greeting Agent",
    instructions="You are a friendly assistant that greets the user and tells a joke.",
)

In [31]:
def fetch_news(country="us", category=None):
    """
    Fetches the top news headlines for a given country and category.

    Args:
        country (str): The country code (default is "us").
        category (str): The news category (optional).

    Returns:
        str: A string containing the top news headlines or an error message.
    """
    base_url = "https://newsapi.org/v2/top-headlines"
    params = {
        "country": country,
        "apiKey": news_api_key
    }

    # Add a category if provided
    if category:
        params["category"] = category

    # Make a GET request to the News API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if the request was successful
    if response.status_code == 200 and 'articles' in data:
        # Extract the titles of the first 10 articles
        headlines = [article['title'] for article in data['articles'][:10]]
        return f"Here are the top news for {country.upper()} ({category or 'all categories'}):\n" + "\n".join(headlines)
    else:
        # Return an error message if the request failed
        return f"Sorry, I couldn't fetch the news at the moment: {data.get('message', 'Unknown error')}"


In [32]:
news_agent = Agent(
    name="News Agent",
    instructions="You provide the top news headlines for a given country and category.",
    functions=[fetch_news]
)

In [33]:
def transfer_to_agent(agent_name):
  agents = {
      "Greeting Agent": greeting_agent,
      "News Agent": news_agent
  }
  return agents[agent_name]

In [34]:
main_agent = Agent(
    name = "Main Agent",
    instructions = """
    You are the main assistant.
    BAsed on the user's request, you decide which specialized agent should handle the task.
    - If the user wants a greeting or a joke, transfer to the Greeting Agent.
    - If the user wants to fetch the news, transfer to the News Agent.
    """,
    functions = [transfer_to_agent]
)

In [35]:
client = Swarm()

In [38]:
response = client.run(
    agent = main_agent,
    messages = [{"role": "user",
                 "content": "tell me news in politics from India"}]
)
print(display(Markdown(response.messages[-1]['content'])))

Here are the top political news headlines from India:

1. **Lok Sabha Passes Women's Reservation Bill**: After extensive debate, the Lok Sabha has passed the historic Women's Reservation Bill, which seeks to reserve 33% of seats in the Lok Sabha and state assemblies for women.

2. **Prime Minister Announces Major Infrastructure Project**: In a recent address, the Prime Minister announced a new major infrastructure project aimed at boosting connectivity and economic growth in the northeastern regions.

3. **Opposition Parties Unite for Nationwide Protest**: Several major opposition parties have decided to unite for a nationwide protest against the government's policies, which they claim are detrimental to the common people.

4. **Parliament Session Extended Amidst Legislative Push**: The current session of Parliament has been extended to allow for the passage of several key bills, including those related to economic reforms and social welfare.

5. **State Elections on the Horizon**: With state elections approaching, political parties have intensified their campaigns, with leaders touring multiple districts to garner support.

For further updates, keep an eye on reputable news sources.

None


In [22]:
response.agent

Agent(name='News Agent', model='gpt-4o', instructions='You provide the top news headlines for a given country and category.', functions=[<function fetch_news at 0x7ebdab717d00>], tool_choice=None, parallel_tool_calls=True)